importing modules and files

In [1]:
import xml.etree.ElementTree as ET 
import re
from Api.apiRequests import getJobListings
from operatingSys.servieces import importXml


Importing and providing the xml data from server to local, then parsing to be usable

In [2]:
url = 'http://localhost:8080/' #source XML address
xmlData = getJobListings(url)



In [3]:
xmlFolderAddress = "xmlFiles/fetchedXml.xml" # the target folder that we have imported our xml response

importXml(xmlFolderAddress,xmlData) #writes the fetched xml to desired address
# removine empty lines
# with open(xmlFolderAddress) as xmlfile:
#     lines = [line for line in xmlfile if line.strip() is not ""]

# with open(xmlFolderAddress, "w") as xmlfile:
#     xmlfile.writelines(lines)

root = ET.parse(xmlFolderAddress).getroot()

In [ ]:
# to validate the RGB color
_rgbstring = re.compile(r'#[a-fA-F0-9]{6}$')

def isrgbcolor(value):
    return bool(_rgbstring.match(value))

Extracting xml tags to variables:
ex:
    jobTitle = 'Python Entwickler '
    responsibilities = '...'

In [18]:
for joblisting in root:
    for i in (joblisting.iter('language')):
        if i.text == 'DE': 
            for elem in joblisting.findall('job_data/jobtitle'):
                jobTitle = elem.text.replace('\n', '')
            for elem in joblisting.findall('job_data/layout_data/images/header'):
                imageUrl = elem.text
            for elem in joblisting.findall('job_data/company_data/companyname'):
                companyName = elem.text.replace('\n', '')
            
            # City list
            locationList = []
            for location in joblisting.findall('job_data/joblocations/location'):
                for cityName in joblisting.findall('job_data/joblocations/location/city'):
                    city = cityName.text
                    city = city.replace(' ', '')
                    city = city.replace('\n', '')

                    if len(joblisting.findall('job_data/joblocations/location/postalcode')) != 0 :
                        for location in location.iter('postalcode'):
                            postalCode = location.text
                            postalCode = postalCode.replace(' ', '')
                            postalCode = postalCode.replace('\n', '')
                            cityAndZip = postalCode +','+ city
                            locationList.append(cityAndZip)
                    else:
                            cityAndZip = city
                            locationList.append(cityAndZip)
                    
            # Contents
            for elem in joblisting.findall('job_data/introduction'):
                introduction = elem.text
                introductionTitle = elem.get('title')
            for elem in joblisting.findall('job_data/responsibilities'):
                responsibilities = elem.text
                responsibilitiesTitle = elem.get('title')
            for elem in joblisting.findall('job_data/qualifications'):
                qualifications = elem.text
                qualificationsTitle = elem.get('title')
            for elem in joblisting.findall('job_data/we_offer'):
                we_offer = elem.text
                we_offerTitle = elem.get('title')
            for elem in joblisting.findall('job_data/contact_data'):
                contact_data = elem.text
                contact_dataTitle = elem.get('title')
            for elem in joblisting.findall('job_data/layout_data/colors/heading_color'):
                heading_color = elem.text.replace('\n', '').replace(' ', '')
            for elem in joblisting.findall('job_data/application_procedure/url'):
                applyUrl = elem.text
            

            
            # making HTML files from above variables
            
            
            #if introductionTitle == 'None':
            #if introductionTitle == 'None':
            #if introductionTitle == 'None':
            #if introductionTitle == 'None':
            # def create_page():
            #     doc = dominate.document(title = jobtitle)
            #     print(doc)
            # create_page()

jobTitle,imageUrl,companyName,locationList,introduction,introductionTitle,responsibilities,responsibilitiesTitle,qualifications,qualificationsTitle,we_offer,we_offerTitle,contact_data,contact_dataTitle,heading_color,applyUrl ===> are extracted as variables
